First create another container (Gold layer) on your adls and create external location on databricks 

You need to add a storage path to your catalog creation command — something like this:

In [0]:
# Create Catalog and Schema
spark.sql("""
CREATE CATALOG IF NOT EXISTS safe_sure_catalog
MANAGED LOCATION 'abfss://gold-layer@sampledemos.dfs.core.windows.net/'
""")

# Switch to the Catalog
spark.sql("USE CATALOG safe_sure_catalog")

# Create Schema (use either DATABASE or SCHEMA — they are synonyms in Databricks)
spark.sql("CREATE SCHEMA IF NOT EXISTS gold")
spark.sql("USE gold")


Create the agent Delta table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.claim (
    ClaimID STRING,
    PolicyID STRING,
    ClaimDate DATE,
    ClaimType STRING,
    Amount DOUBLE,
    Status STRING
)
USING DELTA;


In [0]:
# load the data from the silver layer 

df = spark.read.option("header", "true").csv("abfss://landingzone@sampledemos.dfs.core.windows.net/InsuranceClaim.csv")

from pyspark.sql import functions as F


df_clean = (df
    .withColumn("ClaimDate", F.to_date(F.col("ClaimDate"), "dd/MM/yyyy"))
    .withColumn("ClaimType", F.initcap(F.trim("ClaimType")))
    .withColumn("Status", F.when(F.lower(F.col("Status")) == "approved", "Approved")
                           .when(F.lower(F.col("Status")) == "pending", "Pending")
                           .otherwise("Rejected"))
    .withColumn("Amount", F.col("ClaimAmount").cast("double"))
)

df_clean.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("gold.claim")




In [0]:
%sql
select * from gold.claim